# El Niño in ERA5

## Import packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Time period
start = "1940-01"
stop = None  # None: present

## Define request

In [ ]:
collection_id = "reanalysis-era5-pressure-levels-monthly-means"

request = {
    "format": "grib",
    "product_type": "monthly_averaged_reanalysis",
    "variable": [
        "temperature",
        "u_component_of_wind",
        "relative_humidity",
        "fraction_of_cloud_cover",
        "vertical_velocity",
    ],
    "pressure_level": [
        "100",
        "125",
        "150",
        "175",
        "200",
        "225",
        "250",
        "300",
        "350",
        "400",
        "450",
        "500",
        "550",
        "600",
        "650",
        "700",
        "750",
        "775",
        "800",
        "825",
        "850",
        "875",
        "900",
        "925",
        "950",
        "975",
        "1000",
    ],
    "time": "00:00",
    "area": [5, -170, -5, -120],
}
requests = download.update_request_date(request, start=start, stop=stop)

## Download and transform

In [ ]:
ds = download.download_and_transform(
    collection_id,
    requests,
    transform_func=diagnostics.spatial_weighted_mean,
    chunks={"year": 1, "variable": 1},
)
with xr.set_options(keep_attrs=True):
    ds["plev"] = ds["plev"] / 100
ds["plev"].attrs["units"] = "hPa"

## Compute anomaly

In [ ]:
group = "forecast_reference_time.month"
with xr.set_options(keep_attrs=True):
    ds_anoma = ds.groupby(group) - ds.groupby(group).mean()
for varname, da in ds_anoma.data_vars.items():
    da.attrs["long_name"] += " anomaly"

## Show min and max values

In [ ]:
datasets = []
for reduction in ("min", "max"):
    datasets.append(
        getattr(ds_anoma.sel(plev=slice(1000, 10)), reduction)().expand_dims(
            reduction=[reduction]
        )
    )
df = xr.concat(datasets, "reduction").to_pandas()
df.T

## Plot Hovmöller diagrams

In [ ]:
plot_dict = {
    "t": {"levels": np.arange(-4, 4 + 0.5, 0.5), "cmap": "RdBu_r"},
    "u": {"levels": np.arange(-20, 20 + 1, 1), "cmap": "PuOr"},
    "w": {"levels": np.arange(-0.1, 0.1 + 0.01, 0.01), "cmap": "PuOr"},
    "r": {"levels": np.arange(-40, 40 + 4, 4), "cmap": "BrBG"},
    "cc": {"levels": np.arange(-0.5, 0.5 + 0.05, 0.05), "cmap": "PRGn"},
}
nino_dates = [
    "1951-10-01",
    "1958-01-01",
    "1963-11-01",
    "1965-10-01",
    "1969-01-01",
    "1972-10-01",
    "1982-11-01",
    "1987-08-01",
    "1992-01-01",
    "1997-10-01",
    "2002-11-01",
    "2009-12-01",
    "2015-11-01",
]
nino_dates = xr.DataArray(pd.to_datetime(sorted(nino_dates)))
nino_dates = nino_dates.sel(
    dim_0=slice(
        ds_anoma["forecast_reference_time"].min(),
        ds_anoma["forecast_reference_time"].max(),
    )
)

for time_sel, title in zip(
    [slice(None, None), slice("1995-10-01", "2000-10-01")],
    ["Major El Nino events in the Nino3.4 area", "El Nino 1997-1998"],
):
    fig, axs = plt.subplots(len(ds_anoma.data_vars), figsize=(15, 16))
    ds_to_plot = ds_anoma.sel(forecast_reference_time=time_sel)
    for (varname, kwargs), ax in zip(plot_dict.items(), axs):
        ds_to_plot[varname].plot.contourf(
            y="plev", yincrease=False, extend="both", ax=ax, **kwargs
        )
        for nino_date in nino_dates.sel(dim_0=time_sel).values:
            ax.axvline(nino_date, color="black", linestyle="-", lw=0.5)
    fig.suptitle(title, fontsize=16, x=0.45, y=0.92)